In [72]:
%matplotlib inline
import plotly
from plotly import tools
from fns import *
from syntheticdata import synthetic_generation
plotly.tools.set_credentials_file(username='gpernelle', api_key='4KIdNJBBApIeebdKxyN9')
import plotly.offline as offline

plotly.offline.init_notebook_mode()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


## Training

In [ ]:
trainer = Trainer(batch_size=2, optimizer="adam", filter_size=3, layers=4)
path = trainer.train("./models/unet_trained_mix_dropout-05-clahe30_f32",
                     synth=0,
                     freeze_deep_layers=0,
                     training_array = None,
                     validation_array = None,
                     testing_array = None,
                     training_iters=400, 
                     epochs=300, 
                     dropout=0.5, 
                     restore=True,
                     display_step=1)

100%|██████████| 20/20 [00:02<00:00,  7.28it/s]


Feature shape: (400, 148, 148, 148, 2)
b'\n%/job:localhost/replica:0/task:0/cpu:0\x12\tlocalhost\x1a\x0b_0_Iterator \xa4\xd6\xcb\xd1\xa6\xeb\xb2\xc1\xc7\x01*.N10tensorflow12_GLOBAL__N_116IteratorResourceE'
b'\n%/job:localhost/replica:0/task:0/cpu:0\x12\tlocalhost\x1a\x0b_0_Iterator \xa4\xd6\xcb\xd1\xa6\xeb\xb2\xc1\xc7\x01*.N10tensorflow12_GLOBAL__N_116IteratorResourceE'
(tf.float32, tf.float32)
INFO:tensorflow:Restoring parameters from ./models/unet_trained_mix_dropout-05-clahe30_f32/model 185.cpkt


In [3]:
def dice(logits, labels):
        flat_logits = logits.flatten()
        flat_labels = labels.flatten()
        intersection = np.sum(flat_logits*flat_labels)
        union = np.sum(flat_logits) + np.sum(flat_labels)
        loss = 1 - 2 * intersection / union
        return loss

## Inference

In [4]:
case='72'
image_name = '/home/gp1514/SSD/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00_split2/0%s/case.nrrd'%case
label_name = '/home/gp1514/SSD/preprocessed_data/LabelMapsNEW2_1.00-1.00-1.00/0%s/needles.nrrd'%case

data, options = nrrd.read(image_name)
data = data.astype(np.float32)
data1, options = nrrd.read(image_name.replace('_split2', ''))
data1 = data1.astype(np.float32)

label_data, options = nrrd.read(label_name)

prediction = predict([data1,data], "./models/unet_trained_mix_dropout-05-clahe30_f32")

--------------------------------------------------
Loading and Preparing the data
Original input shape (300, 195, 192)
Padded input shape: (388, 283, 280)
# of parts 7 7 5
number of tiles: 96 
data shape (96, 148, 148, 148)
Original input shape (300, 195, 192)
Padded input shape: (388, 283, 280)
# of parts 7 7 5
number of tiles: 96 
data shape (96, 148, 148, 148)


100%|██████████| 96/96 [00:00<00:00, 729444.17it/s]

--------------------------------------------------
Starting the segmenter.


imgs shape (96, 148, 148, 148, 2)
Feature shape:  (96, 148, 148, 148, 2)
INFO:tensorflow:Restoring parameters from ./models/unet_trained_mix_dropout-05-clahe30_f32/model 41.cpkt


100%|██████████| 96/96 [00:47<00:00,  2.18it/s]


prediction shape: (96, 1, 60, 60, 60, 1)
Merging subvolumes
--------------------------------------------------
Padded input shape: (388, 283, 280)
# of parts 7 7 5
# of subvolumes merged:  96
(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=uint16), array([11206553,    17706,     7169,      277,      264,       18,
              8,        5]))
(300, 195, 192)


In [5]:
dice(prediction, label_data)

0.35283225129955598

In [30]:
islands = post_processing(prediction, min_area=int(100), max_residual=float(4))
# islands.shape

Number of regions: 67
1 1.02826171946 116
2 1.15177152722 297
3 1.12340012298 178
4 1.16889649616 112
5 3.67933554757 2580
6 1.17268402643 941
7 1.15137775692 857
8 1.98433331641 1403
9 1.19710388199 986
10 1.12828942045 927
11 1.18400254281 486
12 1.51766449149 221
13 1.14371138304 499
14 1.19906235038 137
15 1.0440969806 124
16 1.06518561515 156
17 1.14601242062 433
18 1.14039425844 990
19 1.12425107633 384
20 1.1222831301 535
21 1.15644452255 949
22 1.21801716084 511
23 1.1667043662 1089
24 1.16976536829 888
25 1.21132853169 663
26 1.16099200078 797
27 1.13387091809 849
28 1.01131256407 144
29 1.24940084293 933
30 1.11572836494 780
31 1.21372014749 988
32 1.04008536685 125
33 0.993174034858 192


In [95]:
import plotly.plotly as py
import plotly.graph_objs as go
x,y,z = np.where(label_data == 1)
xs,ys,zs = np.where(islands != 0)

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=3,
        line=dict(
            color='green',
            width=2
        ),
        opacity=0.3
    )
)

trace2 = go.Scatter3d(
    x=xs,
    y=ys,
    z=zs,
    mode='markers',
    marker=dict(
        color='red',
        size=3,
        symbol='circle',
        line=dict(
            color='rgb(204, 204, 204)',
            width=1
        ),
        opacity=0.3
    )
)
data = [trace1, trace2]
layout = go.Layout(
    title="72",
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=100
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='72')

<plotly.tools.PlotlyDisplay object>
